# Case Study 1 - Simulated Fibre Data

CRBS VectoRose Bootcamp

Benjamin Z. Rudski <benjamin.rudski@mail.mcgill.ca>

August 6, 2025

In this notebook, we'll explore the basic functionality of VectoRose through the analysis of some artificial fibre data. These fibres have been constructed using [**Dragonfly 3D World**](https://dragonfly.comet.tech/en/products/dragonfly-3d-world), version 2024.1.

## Outline

This notebook shows all the major steps of a standard VectoRose workflow, from loading and cleaning data to analysing.

Here's the outline for this notebook:

1. Sample Data
2. Getting Started with VectoRose
3. Loading Vector Data
5. Constructing Orientation Histograms
6. Exporting Screenshots and Animations
7. Orientation Filtering
8. Computing Directional Statistics
9. Polar Plotting (Optional)

By following along this worked example, you should gain the skills necessary to apply VectoRose to your own data, regardless of the sample under study and the data acquisition methods used.

## Sample Data

Before analysing the vectors, let's take a step back... The vectors reflect the architecture of some real, physical object. Let's take a look at that object in 3D! For starters, let's look at a video showing the simulated fibres and their orientations.

In [ ]:
from IPython.display import Video

Video("../assets/CaseStudy1/RandomCylinders2.mp4", width=720)

Note that the orientation colours correspond to the Cartesian axes, with red representing the *x*-axis, green representing the *y*-axis and blue representing the *z*-axis.

We can also view the fibres interactively in 3D by taking take advantage of the [PyVista](https://docs.pyvista.org/) package. This package is built on the foundations of the [Visualization Toolkit](https://vtk.org/), which is a very fast and powerful visualisation library.

I've provided a mesh showing our artificial fibres in the file [`assets/CaseStudy1/RandomCylinders2.ply`](assets/CaseStudy1/RandomCylinders2.ply).

Let's import PyVista and visualise our structure:

In [ ]:
import pyvista as pv

mesh = pv.read("../assets/CaseStudy1/RandomCylinders2.ply")
mesh.plot()

So, we can see that we have a complicated mess of fibres, going in all sorts of different directions. Now, let's get started with the vectors!

## Getting Started with VectoRose

If you haven't already done so, make sure to install the `vectorose` package. This can be done easily in Jupyter Lab by running the following cell:

In [ ]:
# Install vectorose
# Only run this cell if you don't have vectorose installed.
%pip install vectorose

And, as usual, the first thing we need to do is import our package. To save space, we will give `vectorose` the shorter name `vr` when we import it.

> 📝 **Note**\
> This is just a convention. You can rename the package to anything else meaningful.

In [ ]:
# Your code here to import vectorose


## Loading Vector Data

To study the orientations of the fibres, I've computed an anisotropy field using [**Dragonfly 3D World**](https://dragonfly.comet.tech/en/product-overview/dragonfly-3d-world). These vectors are stored in a *binary NumPy file*, located at [`assets/CaseStudy1/RandomCylinders2_S5_R5_SMOOTH1_EIG_MASKED.npy`](assets/CaseStudy1/RandomCylinders2_S5_R5_SMOOTH1_EIG_MASKED.npy).

To be able to work with vectors, first we need to load them into our Python session. To do this, we're going to use the function [`vr.io.import_vector_field`](https://vectorose.readthedocs.io/en/latest/autoapi/vectorose/io/index.html#vectorose.io.import_vector_field). For more details about the types of files that can contain vector data and how to format vectors in these files, make sure to check out the VectoRose [**User's Guide**](https://vectorose.readthedocs.io/en/latest/users_guide/data_format.html).

In [ ]:
vectors_filename = "../assets/CaseStudy1/RandomCylinders2_S5_R5_SMOOTH1_EIG_MASKED.npy"

# Your code here to load the vectors


We can now see that we have an array containing 18 716 vectors. The first three columns correspond to the spatial `x`, `y`, `z` coordinates, while the last three correspond to the vector `x`, `y`, `z` components.

Now that we have the vectors loaded, we can start performing he analysis. But we need to be careful! We can't analyse the orientations of any vectors with zero magnitude... since they don't have orientation. So, we need to perform a pruning step. We'll call the function [`vr.util.remove_zero_vectors`](https://vectorose.readthedocs.io/en/latest/autoapi/vectorose/util/index.html#vectorose.util.remove_zero_vectors) to perform this step.

In [ ]:
# Your code here to remove zero-vectors


We can see that we now have 18 711 vectors, so we've removed a few vectors with zero magnitude.

There's one last processing step that we'd like to do. Our fibres don't actually have any inherent *direction*. So, we want to make sure that all antiparallel vectors are considered as the same. To do this, we will flip all vectors with a negative `z`-component so that all vectors lie in the upper hemisphere. Since we're doing this to convert vectorial data into axial data, we'll be using the function [`vr.util.convert_vectors_to_axes`](https://vectorose.readthedocs.io/en/latest/autoapi/vectorose/util/index.html#vectorose.util.convert_vectors_to_axes).

In [ ]:
# Your code here to convert vectors to axes


As you can see, all vectors now have a positive `z`-component. All our vectors now lie in the upper hemisphere.

While this will be helpful for computing statistics, it will make the spherical visualisations a bit less exciting. Well, we can duplicate the vectors and create flipped copies on the bottom hemisphere to make sure that we get histograms plotted on the full sphere. We'll use the function [`vr.util.create_symmetric_vectors_from_axes`](https://vectorose.readthedocs.io/en/latest/autoapi/vectorose/util/index.html#vectorose.util.create_symmetric_vectors_from_axes).

In [ ]:
# Your code here to create symmetric vectors


Now you can see that we have twice as many vectors as before! Now that we've done all this pre-processing, we can begin analysing our data!

## Constructing Orientation Histograms

We'll now begin the process of constructing an orientation histogram. Remember that one of the key features of a histogram is the binning. A histogram shows the frequency of the data in each bin. When we're talking about orientations and directions, we need to assign the data to bins on the surface of a sphere.

There are many different ways of defining these bins. One such way involves a (largely) rectangular tiling, based on the work by [Beckers & Beckers (2012)](10.1016/j.comgeo.2012.01.011), known as a *Tregenza sphere*. In VectoRose, we can create different Tregenza spheres with different-sized bins. We'll focus primarly on the [`FineTregenzaSphere`](https://vectorose.readthedocs.io/en/latest/autoapi/vectorose/tregenza_sphere/index.html#vectorose.tregenza_sphere.FineTregenzaSphere). This sphere representation consists of 54 rings, containing a total of 5806 bins.

Let's now create a `FineTregenzaSphere` so that we can compute our histograms:

In [ ]:
# Your code here for generating a FineTregenzaSphere


Now, we can assign our vectors to different orientation bins using the method [`assign_histogram_bins`](https://vectorose.readthedocs.io/en/latest/autoapi/vectorose/sphere_base/index.html#vectorose.sphere_base.SphereBase.assign_histogram_bins). This method returns **two** objects, with the first being the labelled vectors. We'll discuss the second object in another example.

Let's now assign our vectors to bins!

In [ ]:
# Your code here to assign vectors to orientation bins


We can see that our vectors are now a bit different... Instead of a simple array, we have something that looks more like a spreadsheet. This is a `DataFrame`, included in the pandas package. We can see that our vectors are now represented using their spatial locations (**x**, **y**, **z**), as well as their orientations (**phi**, **theta**) and lengths (**magnitude**).

We also have the three histogram-related columns **shell**, **ring**, **bin**. We won't worry about the **shell** right now, since we're only focussing on orientation. The **ring** and the **bin** tell us where in the Tregenza sphere our vectors land. These columns are really important for building our histograms!

Now that we have our data assigned to bins, we need to figure out how many vectors are in each bin to construct a histogram. Thankfully, we don't have to do any of these calculations ourselves! VectoRose takes care of everything. We can construct our orientation histogram using the method [`construct_marginal_orientation_histogram`](https://vectorose.readthedocs.io/en/latest/autoapi/vectorose/sphere_base/index.html#vectorose.sphere_base.SphereBase.construct_marginal_orientation_histogram) defined on the `FineTregenzaSphere`.

In [ ]:
# Your code here to compute the orientation histogram


We now have a single column with fractions represented as decimals. Each row contains the fraction of vectors lying in each face of the sphere. Notice how we have a multi-level index, where we are looking by ring, and then by bin in the ring.

We can make sure that everything makes sense here by checking the sum of all these entries. We expect that the total sum of the fractions should equal 1.

In [ ]:
# Your code here to check the sum of all entries


Yay! It worked!

So, we have all the data that we need to create our histogram, but we still have a small problem... we're looking at a flat list of numbers. But we want a nice visual, 3D spherical histogram.

We can construct this histogram quite easily.

First, we need to create a mesh of the sphere we want to visualise using the method [`create_shell_mesh`](https://vectorose.readthedocs.io/en/latest/autoapi/vectorose/sphere_base/index.html#vectorose.sphere_base.SphereBase.create_shell_mesh).

In [ ]:
# Your code here to construct the histogram mesh


Now that we have the mesh, we must plot it. We could do this directly using PyVista, like we did in the intro. But VectoRose provides a [`SpherePlotter`](https://vectorose.readthedocs.io/en/latest/autoapi/vectorose/plotting/index.html#vectorose.plotting.SpherePlotter) class that helps with some of the plotting tasks that we'll see.

> ⚠️ **Warning**\
> It is super important that after creating the `SpherePlotter`, you **must** call the method [`produce_plot`](https://vectorose.readthedocs.io/en/latest/autoapi/vectorose/plotting/index.html#vectorose.plotting.SpherePlotter.produce_plot). Otherwise, you won't be able to see the histogram.

In [ ]:
# Your code here to produce the SpherePlotter


We did it! Now we have a spherical histogram! We can see bright spots representing the different fibres observed in our dataset.

Confused about the angles? Well, we can add some spherical axes using the [`add_spherical_axes`](https://vectorose.readthedocs.io/en/latest/autoapi/vectorose/plotting/index.html#vectorose.plotting.SpherePlotter.add_spherical_axes) method.

In [ ]:
# Your code here to add spherical axes


Congratulations! You've just created your first orientation histogram plot 🎉!

> 💡 **Tip**\
> Don't lose track of this plotter. We'll come back to it in later sections.

## Exporting Screenshots and Animations

At this point, we have a 3D spherical histogram that is fun to interact with, but it only lives in our Jupyter notebook. If we stop the notebook and reopen it, we no longer have access to our plotter. How can we save our insights for later use?

Good news! With VectoRose, it's easy to save images from plotters, and even to generate videos. In this section, we'll see the basics of these tasks.

### Exporting Screenshots

A picture is worth a thousand words...

VectoRose contains two methods for saving screenshots:

* [`SpherePlotter.export_screenshot`](https://vectorose.readthedocs.io/en/latest/autoapi/vectorose/plotting/index.html#vectorose.plotting.SpherePlotter.export_screenshot) - export raster images of plots, such as `PNG` or `TIFF`.
* [`SpherePlotter.export_graphic`](https://vectorose.readthedocs.io/en/latest/autoapi/vectorose/plotting/index.html#vectorose.plotting.SpherePlotter.export_graphic) - export vector graphics of plots, such as `PDF` or `SVG`.

> ⚠️ **Attention**\
> When exporting to a vector graphics format, the plot is still rendered as a raster image. However, any text becomes easily editable (and does not look pixelated).

Let's see some examples with our previous plot.

In [ ]:
# Your code here to save a screenshot


Let's now take a quick look at our image...

In [ ]:
import matplotlib.pyplot as plt
import PIL

im = PIL.Image.open("simulated_fibres.png")

plt.imshow(im)
plt.show()

In [ ]:
# Your code here to save PDF


These functions both have different parameters that can be set. Make sure to check out the documentation to learn more about all the possibilities.

### Exporting Animations

So... a picture is worth a thousand words... about the side of the sphere that is currently visible!

In a screenshot, we can only see one side of the spherical orientation histogram. What if we want to view the sphere from many different angles?

Well... a video is worth a thousand pictures! We can create animations that show the orientation histogram from many different perspectives. The simplest type of video we can create is a *rotation animation*. We do this using the method [`produce_rotating_video`](https://vectorose.readthedocs.io/en/latest/autoapi/vectorose/plotting/index.html#vectorose.plotting.SpherePlotter.produce_rotating_video).

Make sure to check out the documentation to learn about the various parameters.

The first parameter is the output filename, which must either be a an `mp4` or `gif` file.

In [ ]:
# Your code here to create a rotating animation


And now, let's take a look at the video we've produced.

In [ ]:
Video("rotating_sphere.mp4", html_attributes="loop autoplay controls")

We can now see the sphere rotating about its axis and examine the results more easily.

## Orientation Filtering

Looking at the orientation histogram, we see a bunch of spots. Thinking back to our data, what do these different spots represent?

Well, our data consist of perfectly straight fibres in different orientations. So, it would stand to reason that the different bright spots correspond to fibres in different orientations.

Wouldn't it be nice if we could manually select these different bright spots and then extract the corresponding vectors?

Let's see how we can do this sort of **orientation filtering** with VectoRose.

### Enabling Cell Picking

The first thing we need to do is enable cell picking. We do this by setting the property [`cell_picking_active`](https://vectorose.readthedocs.io/en/latest/autoapi/vectorose/plotting/index.html#vectorose.plotting.SpherePlotter.cell_picking_active) to `True` on the `SpherePlotter`.

In [ ]:
# Your code here to enable cell picking


Now, let's go back up to our sphere plotter. If we **right-click** on a face, we see that it becomes outlined in magenta. If we click again, the outline disappears.

By right-clicking, we can select and deselect cells.

> ⚠️ **Warning**\
> Don't forget, you must **right-click** to select cells.


### Extracting Vectors from Picked Cells

Choose a patch of faces that you are interested in exploring further. Select the cells by right-clicking. Now, we'll see how to extract the vectors that are only aligned with that orientation.

The cells that we've selected are stored in the [`picked_cells`](https://vectorose.readthedocs.io/en/latest/autoapi/vectorose/plotting/index.html#vectorose.plotting.SpherePlotter.cell_picking_active) property of the `SpherePlotter`. Let's look at the cells we've selected.

In [ ]:
# Your code here to get the selected cells


So, we can see for each cell we've selected what the bin, ring and frequency are. Using our `FineTregenzaSphere`, we can extract the vectors falling in those bins using the method [`get_vectors_from_selected_cells`](https://vectorose.readthedocs.io/en/latest/autoapi/vectorose/sphere_base/index.html#vectorose.sphere_base.SphereBase.get_vectors_from_selected_cells).

In [ ]:
# Your code here to extract the vectors

selected_vectors = my_sphere.get_vectors_from_selected_cells(labelled_vectors, selected_cells)

selected_vectors

We can now see that there are much fewer vectors in our selected dataset. We can also use the locations to plot where these vectors lie in our dataset in 3D.

In [ ]:
mesh_plotter = pv.Plotter()
mesh_plotter.add_mesh(mesh, opacity=0.5)

vector_locations = selected_vectors[["x", "y", "z"]].to_numpy()
vector_locations_mesh = pv.PointSet(vector_locations)
mesh_plotter.add_mesh(vector_locations_mesh, color="red", render_points_as_spheres=True)
mesh_plotter.show()

We can now see that most of the vectors we've extracted are contained within a single fibre. So, for clean data, we can easily perform orientation filtering.

## Computing Directional Statistics

So far, we've seen how to visualise collections of vectorial data. Spherical histograms give us a great way to quickly see how our data are distributed. However, sometimes, we want to put numbers on things. Let's start applying some directional statistics to our collection of orientation vectors.

### Preparing Data for Statistical Analysis

Before computing statistics, we need to get our data into the right format. We need to get the vector components in cartesian form. Since we are looking at statistics for pure orientations, we need to consider **unit vectors**. We also need to consider the original, **unduplicated** axes.

We'll create a collection of unit vectors using the function [`vr.util.normalise_vectors`](https://vectorose.readthedocs.io/en/latest/autoapi/vectorose/util/index.html#vectorose.util.normalise_vectors).

In [ ]:
# Your code here to get unit axes


### Computing the Mean Resultant Vector

We can compute a few different statistical measurements. Let's start with computing the mean resultant vector. We can do this using the function [`vr.stats.compute_resultant_vector`](https://vectorose.readthedocs.io/en/latest/autoapi/vectorose/stats/index.html#vectorose.stats.compute_resultant_vector).

In [ ]:
# Your code here to compute the mean resultant vector


We can now see that the mean resultant vector has a fairly short length. This makes sense, given the diverse fibre orientations.

The orientation is fairly close to the sphere pole (think about just how close 4.5° is to the pole). This also makes sense. We're considering **orientations**, not directions, so all our vectors are in the upper hemisphere. So, the **Z**-components can't cancel each other out.

### Computing Woodcock's Shape and Strength Parameters

The mean resultant vector gives us a quick way to assess the level of coalignment of the vectors. But if we want a clearer picture of the overall shape of the distribution, we can compute the **orientation matrix** and Woodcock's **shape and strength parameters** based on the orientation matrix eigenvalues.

We can first compute the orientation matrix eigen-decomposition by calling the function [`vr.stats.compute_orientation_matrix_eigs`](https://vectorose.readthedocs.io/en/latest/autoapi/vectorose/stats/index.html#vectorose.stats.compute_orientation_matrix_eigs). Then, we can compute the shape and strength parameters by calling the function [`compute_orientation_matrix_parameters`](https://vectorose.readthedocs.io/en/latest/autoapi/vectorose/stats/index.html#vectorose.stats.compute_orientation_matrix_parameters).

Let's see how to do it:

In [ ]:
# Your code here to compute Woodcock's shape and strength parameters


The strength parameter is quite low, indicating that the distribution is fairly uniform.

Let's try doing the same thing, but with the vectors we extracted for a single fibre. To convert these to unit vectors, we'll use a different approach. We can convert the filtered vectors `DataFrame` to an array containing unit vectors using the [`convert_vectors_to_cartesian_array`](https://vectorose.readthedocs.io/en/latest/autoapi/vectorose/sphere_base/index.html#vectorose.sphere_base.SphereBase.convert_vectors_to_cartesian_array) method of our sphere representation.

In [ ]:
unit_filtered_vectors = my_sphere.convert_vectors_to_cartesian_array(
    selected_vectors, create_unit_vectors=True
)

eigs, _ = vr.stats.compute_orientation_matrix_eigs(unit_filtered_vectors)
params = vr.stats.compute_orientation_matrix_parameters(eigs)

print(f"Strength parameter: {params.strength_parameter}")
print(f"Shape parameter: {params.shape_parameter}")

So, when we look within a narrow window, we get a very strong cluster at a single location.

So... What else can we do with these filtered vectors?

### Estimating VMF Parameters

Well, since we have vectors around a single direction, we could estimate the parameters for a von Mises-Fisher distribution. Recall that this distribution is similar to the Gaussian distribution. We have two parameters:

* $\mu$ - the mean direction
* $\kappa$ - the concentration parameter (higher = more compact cluster)

In VectoRose, we can estimate these parameters using the function [`vr.stats.fit_fisher_vonmises_distribution`](https://vectorose.readthedocs.io/en/latest/autoapi/vectorose/stats/index.html#vectorose.stats.fit_fisher_vonmises_distribution). Let's do this for our original data and for our filtered data.

In [ ]:
# Your code here to compute the VMF parameters for original data


In [ ]:
# Your code here to compute the VMF parameters for the filtered data


These results shouldn't be very surprising.

Since we have fibres aligned in many different orientations, we get a very low concentration parameter in the first example.

Since we then selected vectors that are very closely aligned with each other, it makes sense that the concentration parameter is much higher.

> ⚠️ **Warning**\
> This was an artificial example just to demonstrate differences in the results. In practice, doing a calculation like this introduces **bias** into your dataset. We knew *a priori* that we would get a high concentration parameter simply by virtue of selecting vectors that are co-aligned in a small neighbourhood. Think very carefully about your statistical workflow and whether fitlering will unnecessarily bias your results.


There are other statistical routines implemented in VectoRose. Make sure to check out the documentation for the [`vr.stats`](https://vectorose.readthedocs.io/en/latest/autoapi/vectorose/stats/index.html) module, as well as the [Users' Guide](https://vectorose.readthedocs.io/en/latest/users_guide/statistics.html) page on statistics.

## Polar Plotting (Optional)

Up until now, we've been focussing on looking at orientation in 3D. But, sometimes, you may want to look at the orientation a bit differently...

Orientation in 3D can be defined in two different ways:

* Cartesian coordinates: the familiar *x*, *y*, *z* system
* Spherical coordinates: a length (*r*) and two angles ($\phi$, $\theta$)

These two angles represent different aspects of the orientation:

* $\phi$ - the **angle of inclination**, measured downwards from the positive *z*-axis (0°) to the *xy*-plane (90°, for axial data) or the negative *z*-axis (180°, for vectorial data).
* $\theta$ - the **azimuthal angle**, measured in the *xy*-plane as the clockwise angle from the positive *y*-axis (0° and 360°).

While these two angles are used to define orientations in 3D, we can also construct **polar histograms** of each angle separately. In VectoRose, most of the heavy lifting is done by the [`vr.polar_data`](https://vectorose.readthedocs.io/en/latest/autoapi/vectorose/polar_data/index.html) module.

We'll see how to assign vectors to angular bins and then how to visualise the resulting polar histograms.

### Assigning Polar Histogram Bins

To assign polar histogram bins, we need to create a [`PolarDiscretiser`](https://vectorose.readthedocs.io/en/latest/autoapi/vectorose/polar_data/index.html#vectorose.polar_data.PolarDiscretiser). Similar to our `FineTregenzaSphere` from earlier, this object is responsible for labelling vectors and computing histograms. Based on the documentation, we need to provide:

* the number of bins we want for the $\phi$ histogram
* the number of bins we want for the $\theta$ histogram
* an indication of whether the data are axial

Let's now create a `PolarDiscretiser` for our simulated fibre data. We'll set there to be 10 bins along the $\phi$ axis and 36 bins along the $\theta$ axis, so that our bins each contain 10 degrees.

> ⚠️ **Attention**\
> For technical reasons, we need an extra bin in the $\phi$ axis to account for any vectors that are perfectly aligned with the last bin edge. Though this may be rare in real-world data, it can occur in simulated vectors.


In [ ]:
# Your code here to construct a polar discretiser


Now, like with our Tregenza sphere earlier, we can use the method [`assign_histogram_bins`](https://vectorose.readthedocs.io/en/latest/autoapi/vectorose/polar_data/index.html#vectorose.polar_data.PolarDiscretiser.assign_histogram_bins). Unlike when working with the 3D spherical plots, we will work with the **unduplicated axes**.

In [ ]:
# Your code here to assign bins to the vectors


Like before, we now have a `DataFrame` containing the vectors and their associated bins.

### Computing Polar Histograms

We can now gather the bin information and construct $\phi$ and $\theta$ histograms using the `PolarDiscretiser` methods [`construct_phi_histogram`](https://vectorose.readthedocs.io/en/latest/autoapi/vectorose/polar_data/index.html#vectorose.polar_data.PolarDiscretiser.construct_phi_histogram) and [`construct_theta_histogram`](https://vectorose.readthedocs.io/en/latest/autoapi/vectorose/polar_data/index.html#vectorose.polar_data.PolarDiscretiser.construct_theta_histogram).

In [ ]:
# Your code here to construct the phi histogram


In [ ]:
# Your code here to construct the theta histogram


So, we can now see how many vectors fall into each polar bin, as both counts and frequencies.

But... this isn't **yet** a plot...

### Visualising Polar Histograms

The final step here is to create visual polar plots of the histogram data. While we could manually do this with Matplotlib, VectoRose provides some helpful functions to make the process easier.

We can easily create a single figure with both the $\phi$ and $\theta$ histograms using the function [`vr.plotting.produce_phi_theta_polar_histogram_plots`](https://vectorose.readthedocs.io/en/latest/autoapi/vectorose/plotting/index.html#vectorose.plotting.produce_phi_theta_polar_histogram_plots).

While this function has lots of different parameters, the most important ones for us are the first two: the $\phi$ histogram data and the $\theta$ histogram data.

Another helpful parameter is `use_counts`, where we can set whether we want the histogram to show counts or fractions.

Let's use this function to visualise our polar histograms.

In [ ]:
# Your code here to visualise the polar histograms


Looking at these plots, we can see that many of the vectors in our simulated fibres are inclined to near the *xy*-plane, and have a variety of different headings.

## Conclusion

In this notebook, we've seen the basics of using VectoRose to analyse vectors computed based on a simulated fibre system. Here's a summary of what we've covered:

* We can use VectoRose to construct **spherical orientation histograms**.
* We can easily capture **screenshots** and **animations** of these 3D plots.
* We can **interactively** select specific sphere faces to **filter vectors by orientation**.
* We can compute **directional statistics** to gain additional quantitative insights into the data.
* We can construct **polar plots** to get a broader overview of the orientations present in the data.

At this point, we've covered many of the basics of VectoRose... but there's an important idea that we haven't seen yet.

What if your vectors have a non-unit length, and you want to look at more than just the orientation/direction?

The answer is coming up... in the next notebook!

## Extra Practice

Want extra practice using VectoRose? Here's another simulated fibre system, along with its orientation field. The orientation field can be found in [`../assets/CaseStudy1/RandomCylinders3_R5_S5_SMOOTH1_EIG_MASKED.npy`](../assets/CaseStudy1/RandomCylinders3_R5_S5_SMOOTH1_EIG_MASKED.npy) and the mesh is found in [`../assets/CaseStudy1/RandomCylinders3.ply`](../assets/CaseStudy1/RandomCylinders3.ply). Try to redo the steps that we did, but with different data. Look at the orientation patterns. Were these fibres generated in a similar or different way from the ones we considered in this notebook?

In [ ]:
Video("../assets/CaseStudy1/RandomCylinders3.mp4", width=720)

In [ ]:
# Your analysis code here
